In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from prettytable import PrettyTable
from datetime import datetime, timedelta
from tqdm import tqdm
import statsmodels.api as sm
import yfinance as yf
import numpy as np

In [55]:
IPCA=4
SELIC=12.75
ANO_DIAS_UTEIS=252

In [65]:
acoes=["DIVO11.SA","MGLU3.SA", "GGBR4.SA"]
mercado=["^BVSP"]

tickers= acoes + mercado
data = yf.download(tickers, start="2018-1-1")["Adj Close"]
data_indice=data[tickers[-1]]
data=data[tickers[:-1]]
data

[*********************100%%**********************]  4 of 4 completed


,DIVO11.SA,MGLU3.SA,GGBR4.SA
Date,,,
2018-01-02,41.820000,2.390916,8.705924
2018-01-03,41.880001,2.360765,8.692501
2018-01-04,42.310001,2.417448,9.162364
2018-01-05,42.450001,2.369811,9.048257
2018-01-08,42.450001,2.370112,9.363735
...,...,...,...
2023-10-03,79.000000,1.840000,23.290001
2023-10-04,78.669998,1.890000,23.000000
2023-10-05,78.339996,1.810000,22.670000


In [66]:
retorno_diario = data.pct_change()
retorno_anual = retorno_diario.mean() * ANO_DIAS_UTEIS
cov_diaria = retorno_diario.cov()
cov_anual = cov_diaria * ANO_DIAS_UTEIS

In [67]:
retorno_carteira = []
peso_acoes = []
volatilidade_carteira = []
sharpe_ratio = []

In [68]:
# vamos usar uma simulação aleatória
numero_acoes = len(ativos)
numero_carteiras = 100000

np.random.seed(101)

In [69]:
peso = np.random.random(numero_acoes)
peso /= np.sum(peso)
retorno_anual

DIVO11.SA    0.138031
MGLU3.SA     0.149375
GGBR4.SA     0.259512
dtype: float64

In [70]:
# vamos fazer um for loop para preencher as lista que criamos anteriormente
for cada_carteira in range(numero_carteiras):
    # vamos dar um peso aleatório para cada ação dentro de cada carteira
    peso = np.random.random(numero_acoes)
    peso /= np.sum(peso)
    # vamos calcular o retorno das carteiras
    retorno = np.dot(peso, retorno_anual)
    # vamos calcular a volatilidade das carteiras
    volatilidade = np.sqrt(np.dot(peso.T, np.dot(cov_anual, peso)))
    # vamos calcular o índice de Sharpe de cada carteira
    sharpe = retorno / volatilidade
    # aqui nós usamos o método apend para incluir cada carteira nas listas criadas anteriormente
    sharpe_ratio.append(sharpe)
    retorno_carteira.append(retorno)
    volatilidade_carteira.append(volatilidade)
    peso_acoes.append(peso)

In [71]:
carteira = {'Retorno': retorno_carteira,
             'Volatilidade': volatilidade_carteira,
             'Sharpe Ratio': sharpe_ratio}

for contar,acao in enumerate(acoes):
    carteira[acao+' Peso'] = [Peso[contar] for Peso in peso_acoes]

# vamos transformar nosso dicionário em um dataframe
df = pd.DataFrame(carteira)

# vamos nomear as colunas do novo dataframe
colunas = ['Retorno', 'Volatilidade', 'Sharpe Ratio'] + [acao+' Peso' for acao in acoes]
df = df[colunas]

In [73]:
# vamos identificar as variáveis de interesse
menor_volatilidade = df['Volatilidade'].min()
maior_sharpe = df['Sharpe Ratio'].max()

# vamos identificar os dois principais portfolios
carteira_sharpe = df.loc[df['Sharpe Ratio'] == maior_sharpe]
carteira_min_variancia = df.loc[df['Volatilidade'] == menor_volatilidade]

In [83]:
print("Essa é a carteira de Mínima Variância:", '\n', carteira_min_variancia.T*100)
print()
print("Essa é a carteira com maior Sharpe Ratio:", '\n', carteira_sharpe.T*100)

Essa é a carteira de Mínima Variância: 
                     36749
Retorno         13.836244
Volatilidade    22.625904
Sharpe Ratio    61.152223
DIVO11.SA Peso  99.503279
MGLU3.SA Peso    0.247080
GGBR4.SA Peso    0.249641

Essa é a carteira com maior Sharpe Ratio: 
                     98058
Retorno         17.873619
Volatilidade    26.097288
Sharpe Ratio    68.488416
DIVO11.SA Peso  66.429794
MGLU3.SA Peso    0.069215
GGBR4.SA Peso   33.500991


In [82]:
print("Essa é a carteira de Mínima Variância:", '\n', carteira_min_variancia.T*100)
print()
print("Essa é a carteira com maior Sharpe Ratio:", '\n', carteira_sharpe.T*100)

Essa é a carteira de Mínima Variância: 
                     36749
Retorno         13.836244
Volatilidade    22.625904
Sharpe Ratio    61.152223
DIVO11.SA Peso  99.503279
MGLU3.SA Peso    0.247080
GGBR4.SA Peso    0.249641

Essa é a carteira com maior Sharpe Ratio: 
                     98058
Retorno         17.873619
Volatilidade    26.097288
Sharpe Ratio    68.488416
DIVO11.SA Peso  66.429794
MGLU3.SA Peso    0.069215
GGBR4.SA Peso   33.500991
